# Day 17: Chronospatial Computer

https://adventofcode.com/2024/day/17

## --- Part One ---

In [ ]:
import numpy as np
import math
import re

file = 'input.txt'
# file = 'sample.txt'
# file = 'sample2.txt'

# init vars
program = []
regA = 0
regB = 0
regC = 0
results = []

# open file & load content
with open(file, 'r') as f:
    [a, b] = f.read().split('\n\n')
    program = list(map(int,re.findall(r"\d+",b)))
    register = list(map(int,re.findall(r"\d+",a)))
    regA,regB,regC = register

# Combo operands 0 through 3 represent literal values 0 through 3.
# Combo operand 4 represents the value of register A.
# Combo operand 5 represents the value of register B.
# Combo operand 6 represents the value of register C.
# Combo operand 7 is reserved and will not appear in valid programs.

lookup = {0:0,1:1,2:2,3:3,4:regA,5:regB,6:regC}

def get_reg(letter):
    if letter == 'A':
        return lookup[4]
    elif letter == 'B':
        return lookup[5]
    elif letter == 'C':
        return lookup[6]
    else:
        raise

def set_reg(letter, value):
    if letter == 'A':
        lookup[4] = value
    elif letter == 'B':
        lookup[5] = value
    elif letter == 'C':
        lookup[6] = value
    else:
        raise

# The adv instruction (opcode 0) performs division. The numerator is the value in the A register. The denominator is found by raising 2 to the power of the instruction's combo operand. (So, an operand of 2 would divide A by 4 (2^2); an operand of 5 would divide A by 2^B.) The result of the division operation is truncated to an integer and then written to the A register.
def adv(operand):
    combo = lookup[operand]
    numerator = get_reg('A')
    denominator = math.pow(2,combo)
    result = numerator/denominator
    # print('num', numerator, 'den', denominator, 'num/den', math.floor(result))
    set_reg('A', math.floor(result))

# The bxl instruction (opcode 1) calculates the bitwise XOR of register B and the instruction's literal operand, then stores the result in register B.
def bxl(operand):
    result = get_reg('B')^operand
    set_reg('B', result)

# The bst instruction (opcode 2) calculates the value of its combo operand modulo 8 (thereby keeping only its lowest 3 bits), then writes that value to the B register.
def bst(operand):
    combo = lookup[operand]
    result = combo%8
    set_reg('B', result)

# The jnz instruction (opcode 3) does nothing if the A register is 0. However, if the A register is not zero, it jumps by setting the instruction pointer to the value of its literal operand; if this instruction jumps, the instruction pointer is not increased by 2 after this instruction.
def jnz(operand):
    if get_reg('A') == 0:
        pass  
    else:
        return operand

# The bxc instruction (opcode 4) calculates the bitwise XOR of register B and register C, then stores the result in register B. (For legacy reasons, this instruction reads an operand but ignores it.)
def bxc(operand):
    result = get_reg('B')^get_reg('C')
    set_reg('B', result)

# The out instruction (opcode 5) calculates the value of its combo operand modulo 8, then outputs that value. (If a program outputs multiple values, they are separated by commas.)
def out(operand):
    combo = lookup[operand]
    results.append(combo%8)

# The bdv instruction (opcode 6) works exactly like the adv instruction except that the result is stored in the B register. (The numerator is still read from the A register.)
def bdv(operand):
    combo = lookup[operand]
    numerator = get_reg('A')
    denominator = math.pow(2,combo)
    result = numerator/denominator
    set_reg('B', math.floor(result))

# The cdv instruction (opcode 7) works exactly like the adv instruction except that the result is stored in the C register. (The numerator is still read from the A register.)
def cdv(operand):
    combo = lookup[operand]
    numerator = get_reg('A')
    denominator = math.pow(2,combo)
    result = numerator/denominator
    set_reg('C', math.floor(result))


def get_function(n):
    if n == 0:
        return adv
    elif n == 1:
        return bxl
    elif n == 2:
        return bst
    elif n == 3:
        return jnz
    elif n == 4:
        return bxc
    elif n == 5:
        return out
    elif n == 6:
        return bdv
    elif n == 7:
        return cdv
    else:
        # it shouldnt get here
        raise

pointer = 0
while pointer < len(program):
    opcode = program[pointer]
    operand = program[pointer+1]
    # print(opcode,operand, get_reg('B'))
    pointer_move = get_function(opcode)(operand)
    # print(pointer, opcode, operand, pointer_move, lookup)
    if pointer_move != None:
        pointer = pointer_move
    else:
        pointer +=2
    # print('---')

print('Answer to part 1:', ','.join([str(i) for i in results]))

## --- Part Two ---

In [ ]:
""" 
--------
2: Sets value of B = valA % 8 
1: Sets value of B = valB ^ 1 
7: Sets value of C = valA % 8
1: Sets value of B = valB ^ 5 
4: Sets value of B = valB ^ valC
-------------------
"""

original_A = regA
MIN_CHECK = int(math.pow(8,15))
MAX_CHECK = int(math.pow(8,16))

for i in range(MIN_CHECK, MIN_CHECK+5):
    vA = i
    vB = 0
    vC = 0
    # print(vA, vA%8)
    vB = vA % 8
    # print(vB)
    vB = vB^1
    # print(vB)
    vB = vB ^ 5 # 4 bigger than vA%8
    # print(vB)
    vC = math.floor(vA/(math.pow(2,vB)))
    vB = vB ^ vC
    prevB = vB
    vB = vB % 8


In [7]:
program_reversed = list(reversed(program))


MIN_CHECK = int(math.pow(8,15))
MAX_CHECK = int(math.pow(8,16))

outcomes = []
for i in range(MIN_CHECK, MAX_CHECK,50000000):
    outcome = []
    vA = i
    while vA > 0:
        # print(vA, vA%8)
        vB = ((vA % 8)^1)^5
        # print(vB)
        vC = math.floor(vA/(math.pow(2,vB)))
        vB = vB ^ vC
        digit = vB % 8
        outcome.append(digit)
        vA = int(math.floor(vA/8)) 
    if outcome[-1] == 0:
        outcomes.append(vA)
print(len(outcomes))

0


In [6]:
# lists_with_last_zero = [lst for lst in outcomes if lst[-1] == 0]
# print(len(lists_with_last_zero))

print(outcomes[0])

a = [[1,2,3],[4,5,6],[3,3,3],[7,7,7]]
print([lst for lst in a if lst[-1] == 3])

math.log10(1000000000000000)
math.pow(8,15)

IndexError: list index out of range